In [1]:
%pylab inline
#from JSAnimation import IPython_display
from matplotlib import animation
from __future__ import print_function
import time


from channels import *
from compartment import *
from membrane import *
from params import *

from csdmodel1d import *

import numpy as np
import scipy as sp
from scipy.integrate import ode

from ProgressBar import *

import logging
logging.getLogger("FFC").setLevel(logging.ERROR) 

Populating the interactive namespace from numpy and matplotlib


In [2]:
model = CSDModelInterval(N=40,dx=5e-6) # define the model

# Define the compartments, and the membranes
ecs = Compartment("ecs")
neuron = CellCompartment("neuron",density = 2e5) # 2e5 neurons per meter, 4e10 per sq meter
glia = CellCompartment("glia",density = 2e5) #2e5 glia per meter
#neuron = CellCompartment("neuron",density = 4e10)
#glia = CellCompartment("glia",density = 4e10)

neuron_mem = Membrane("neuronal",inside=neuron,outside=ecs,Cm=Cn,phi_m=-70e-3)
glial_mem = Membrane("glial",inside=glia,outside=ecs,Cm=Cg,phi_m=-85e-3)

# Add the compartments to the model
model.addCompartment(ecs,fraction=0.2) # ECS take 20% of the total volume
model.addCompartment(neuron,fraction=0.4) # Neurons take up 40% of the total volume
model.addCompartment(glia,fraction=0.4) # Neurons take up 40% of the total volume

# Add ion species
ecs.addSpecies(K,Ke0,name='K_e')
ecs.addSpecies(Cl,Cle0,name='Cl_e')
ecs.addSpecies(Na,Nae0,name='Na_e')
ecs.addSpecies(Ca,Cae0,name='Ca_e')

neuron.addSpecies(K,Ki0,0,'K_n')
neuron.addSpecies(Na,Nai0,0,'Na_n')
neuron.addSpecies(Cl,Cli0,0,'Cl_n')
neuron.addSpecies(Ca,Cai0,0,'Ca_n')

glia.addSpecies(K,Ki0,name='K_g')
glia.addSpecies(Na,Nai0,name='Na_g')
glia.addSpecies(Cl,Cli0,name='Cl_g')
glia.addSpecies(Ca,Cai0,0,'Ca_g')

# add channels
neuron_mem.addChannel(NaTChannel(),100.) # 100 per neuron?
neuron_mem.addChannel(KDRChannel(),100.) # number of channels per neuron
#neuron_mem.addChannel(KIRChannel(),100.)
neuron_mem.addChannel(NMDAChannel(),300.)
neuron_mem.addChannel(PMCAPump(),500) # PMCA pump
neuron_mem.addChannel(NaCaExchangePump(),500) # sodium-calcium exchanger
neuron_mem.addChannel(NaKATPasePump(),500.) # 5000 ATPase per neuron
neuron_mem.addChannel(NonSpecificChlorideChannel(-0.07),4000)

#glial_mem.addChannel(KIRChannel(),100) # KIR Channel
glial_mem.addChannel(NaKATPasePump(),500.) # 5000 ATPase per neuron
glial_mem.addChannel(PMCAPump(),500)
glial_mem.addChannel(NonSpecificChlorideChannel(-0.085),4000)

model.addMembrane(neuron_mem)
model.addMembrane(glial_mem)

neuron_mem.addLeakChannels()
neuron.balanceWith(ecs)
glial_mem.addLeakChannels()
glia.balanceWith(ecs)

model.assembleSystem()


Ion: K+, g_leak: 1.38E-02
Ion: Cl-, g_leak: 0.00E+00
Ion: Na+, g_leak: 3.95E-03
Ion: Ca2+, g_leak: 1.96E-05
Ion: K+, g_leak: 3.07E-02
Adding a leak channel does nothing to balance Cl-
Ion: Na+, g_leak: 3.62E-03
Ion: Ca2+, g_leak: 1.85E-05


We're going to poke some holes into the membranes in order to instigate CSD

In [3]:
#Hole method for initiation - very slow!!
neuron_hole = HoleChannel([K,Na,Cl],1e-4)
density = np.zeros(model.N)
density[0:3] = 10
neuron_mem.addChannel(neuron_hole,density)

glial_hole = HoleChannel([K,Na,Cl],1e-4)
glial_mem.addChannel(glial_hole,density)



In [4]:
t_old = 0
model.odesolver.t = 0
dt1 = 0 # timescale separation
t_old1 = 0
sol = []
t = []
meanflux = 0.0
sol.append(ecs.value(K))
t.append(0)
start_time = time.time()

In [5]:
t_old = 0
model.odesolver.t = 0
dt1 = 0 # timescale separation
t_old1 = 0
sol = []
t = []
meanflux = 0.0
sol.append(ecs.value(K))
t.append(0)
start_time = time.time()

print( "%s\t\t%s\t%s\t%s, %s, %s, %s" % ('time', 'V_n','V_g', 'K_e', 'K_n','K_g','Cl_e'))
print( "%.3fs\t%.2fmV%.2fmV, %.2fmM, %.2fmM, %.2fmM, %.2fmM" % (model.odesolver.t, 1e3*neuron_mem.phi()[0], 1e3*glial_mem.phi()[0], 1e3*ecs.value(K)[0], 1e3*neuron.value(K)[0], 1e3*glia.value(K)[0], 1e3*ecs.value(Cl)[0]), end='\r')

while model.odesolver.successful() and model.odesolver.t < 180.0:
    y = model.odesolver.integrate(model.odesolver.t+5e-3)
    print( "%.3fs\t%.2fmV,%.2fmV, %.2fmM, %.2fmM, %.2fmM, %.2fmM" % (model.odesolver.t, 1e3*neuron_mem.phi(y)[0], 1e3*glial_mem.phi(y)[0], 1e3*ecs.value(K,y)[0], 1e3*neuron.value(K,y)[0], 1e3*glia.value(K,y)[0], 1e3*ecs.value(Cl,y)[0]), end='\r')
    sol.append(ecs.value(K,y))
    t.append(model.odesolver.t)
    
print( "%.3fs\t%.2fmV%.2fmV, %.2fmM, %.2fmM, %.2fmM, %.2fmM" % (model.odesolver.t, 1e3*neuron_mem.phi()[0], 1e3*glial_mem.phi()[0], 1e3*ecs.value(K)[0], 1e3*neuron.value(K)[0], 1e3*glia.value(K)[0], 1e3*ecs.value(Cl)[0]), end='\r')
elapsed_time = time.time() - start_time
print( "%.3fs\t%.2fmV%.2fmV, %.2fmM, %.2fmM, %.2fmM, %.2fmM" % (model.odesolver.t, 1e3*neuron_mem.phi()[0], 1e3*glial_mem.phi()[0], 1e3*ecs.value(K)[0], 1e3*neuron.value(K)[0], 1e3*glia.value(K)[0], 1e3*ecs.value(Cl)[0]))
print("Elapsed time %.3f" % (elapsed_time) )


time		V_n	V_g	K_e, K_n, K_g, Cl_e
Traceback (most recent call last):
  File "/usr/local/lib/python2.7/site-packages/IPython/core/ultratb.py", line 970, in get_records
    return _fixed_getinnerframes(etb, number_of_lines_of_context, tb_offset)
  File "/usr/local/lib/python2.7/site-packages/IPython/core/ultratb.py", line 233, in wrapped
    return f(*args, **kwargs)
  File "/usr/local/lib/python2.7/site-packages/IPython/core/ultratb.py", line 267, in _fixed_getinnerframes
    records = fix_frame_records_filenames(inspect.getinnerframes(etb, context))
  File "/System/Library/Frameworks/Python.framework/Versions/2.7/lib/python2.7/inspect.py", line 1044, in getinnerframes
    framelist.append((tb.tb_frame,) + getframeinfo(tb, context))
  File "/System/Library/Frameworks/Python.framework/Versions/2.7/lib/python2.7/inspect.py", line 1004, in getframeinfo
    filename = getsourcefile(frame) or getfile(frame)
  File "/System/Library/Frameworks/Python.framework/Versions/2.7/lib/python2.7/inspec

ERROR: Internal Python error in the inspect module.
Below is the traceback from this internal error.


Unfortunately, your original traceback can not be constructed.



TypeError: 'NoneType' object is not iterable

In [ ]:
sol

In [ ]:
vfrac=model.volumefractions(y)

In [ ]:
vfrac